# CS4765/6765 Assingment 1: Spelling Correction

**Due 29 September**

In this assignment you will implement a bigram language model and stupid backoff and apply them for the task of spelling correction. (I've taken care of the task of spelling error detection for you.) You will compare your models with a unigram language model.

Read through this notebook in its entirety before getting started. You should only make changes / write code in parts of the notebook where the instructions ask you to do so. These are indicated with TODO throughout.



## Data

The starter code you have been provided with handles reading the data and tokenization for you. This description is provided to help you to understand how the starter code works and what it is doing.

I've provided you with the following additional files for this assignment:

- `dev.txt` This is development data that you can use to evaluate your models before submitting them.

  This data is a collection of errors in essays by students, taken from the Holbrook Corpus, a well-known spelling error corpus.

  Each line of this file represents a sentence that includes exactly 1 spelling error, which has already been identified for you. Each line of this file consists of an integer, followed by a tab, followed by a sentence. The integer represents the (zero-based) index of the token that has been identified as a spelling error in the sentence. For example, in the following sentence:

  `2   <s> in tow minutes she was back as jane came in the door he hit her on the back of the head she fell to the ground </s>`

  the token at index 2 (tow) is a spelling error. Your job is to correct the indicated spelling errors.

  All of the errors that have been identified for correction, and their corrections, consist entirely of alphabetic characters; additionally, the correction is always within edit distance one of the error. (The spelling error detection is imperfect. If you do find other spelling errors in the dataset, you should not attempt to correct them.)

  I have applied a simple (regex-based) tokenizer to the sentences, and have eliminated most punctuation.

  For this assignment we will tokenize the sentences based on whitespace. I.e., eliminate the initial number and tab, and then split the remaining string based on single whitespace characters. (If you look closely at the data, you’ll see that this tokenization strategy is imperfect. That’s OK for this assignment. Just treat whatever you get from splitting on whitespace as tokens, even if there are some oddities.) The sentences have already been padded with special tokens marking the beginning and end of sentences (`<s>` and `</s>`).

  Note that the starter code you have been provided with handles reading the data and tokenization for you. You should not modify these parts of the starter code.

- `dev.keys.txt` This file has one word per line; each line is the correction to the spelling error on the corresponding line in `dev.txt`. You will use this file for evaluating your spelling corrector during development.

- `test.txt` and `test.keys.txt` This is test data, taken from the same source as the development data, and in the same format. You will use this data for final evaluation of your spelling corrector.

- `corpus.txt` A sample of sentences from the Brown Corpus, tokenized in the same manner as `dev.txt`. You will use this corpus to estimate your language models. (This is a rather small corpus, only about 600<i>k</i> words. In practice you would use a much larger corpus to estimate a language model. However, we’re using a small corpus here to keep the computation manageable.)



## Models

We will use the following model for spelling correction. For a given misspelling $x$, the system's prediction of the correct spelling, $\hat{w}$, is computed as follows:

\begin{equation}
\hat{w} = \mathrm{argmax}_{w \in C} P(w) \tag{1}
\end{equation}

where $P(w)$ is the language model, and $C$ is a set of candidate corrections for the misspelling $x$.


### Unigram language model

The starter code provides a unigram language model. This model does not use any smoothing. Any word that doesn't occur in the training data has probability 0.

### Bigram language model

Implement a bigram language model with Laplace smoothing. Treat any unknown word as a single word (e.g., UNK) which appears in the training data with frequency 0. (Be sure to account for this UNK type when determining the size of the vocabulary.)

Note that in Equation 1 we write $P(w)$ for the language model. Really, though, we’re interested in the probability of the entire sentence, with $w$ replacing $x$. For a bigram language model, a given word instance participates in two bigrams, one with the word before it and one with the word after it. To take this into consideration, we will compute $P(w)$ as below:

\begin{equation}
P(w) ≈ P(w_i|w_{i−1})P(w_{i+1}|w_i) \textrm{ where here } w \textrm{ is the word at position } i \ (\textrm{i.e.}, w_i).
\end{equation}

### Stupid Backoff

Implement stupid backoff as described in Section 3.6.4 of the textbook. In our case, we will only consider up to the case of bigrams, and so, following the notation in the textbook, we will implement this as follows:

\begin{equation}
S(w_i|w_{i-1}) = \begin{cases}
\frac{\textrm{count}(w_{i-1} w_i)}{\textrm{count}(w_{i-1})} & \textrm{if count}(w_{i-1} w_i) > 0 \\
\lambda \frac{\textrm{count}(w_i)}{N} & \textrm{otherwise}
\end{cases}
\end{equation}

where $N$ is the number of token instances (i.e., similarly to the case of the unigram language model).

To use stupid backoff in our spelling corrector, we will compute $P(w)$ as below:

\begin{equation}
P(w) ≈ S(w_i|w_{i−1})S(w_{i+1}|w_i) \textrm{ where here } w \textrm{ is the word at position } i \ (\textrm{i.e.}, w_i).
\end{equation}


### Candidate Corrections

For a given misspelling $x$, the set of candidate corrections is all
words that are within edit distance 1 of $x$ and in-vocabulary; the
vocabulary here is all words (types) in the training corpus
(`corpus.txt`). The edit operations are insertion, deletion,
substitution, and transposition.

Note that the class `CandidateModel` in the starter code below takes care of determining the set of candidate corrections $C$ for a given misspelling $x$ by enumerating all in-vocabulary words that can be arrived at by applying an edit operation to $x$. (An alternative to find all in-vocabulary words within edit distance 1 of $x$ would be to compute the edit distance between $x$ and each word in the vocabulary; however, this approach would tend to be slower.)

In [3]:
# A model of candidate in-vocabulary corrections for a spelling error.
# See below for an example of how to use it.
class CandidateModel:
    def __init__(self, train_corpus_fname):
        self.ALPHABET = 'abcdefghijklmnopqrstuvwxyz'
        self.vocabulary = set()
        for line in open(train_corpus_fname):
            line = line.split()
            # Ignore start and end of sentence markers
            words = line[1:len(line) - 1]
            for w in words:
                self.vocabulary.add(w)

    def delete_edits(self, w):
        # Return the set of strings that can be formed by applying one
        # delete operation to word w
        result = []
        for i in range(len(w)):
            candidate = w[:i] + w[i+1:]
            result.append(candidate)
        return result

    def insert_edits(self, w):
        result = []
        for i in range(len(w) + 1):
            for c in self.ALPHABET:
                candidate = w[:i] + c + w[i:]
                result.append(candidate)
        return result

    def transpose_edits(self, w):
        result = []
        for i in range(1, len(w)):
            transposed_letters = w[i] + w[i-1]
            candidate = w[:i-1] + transposed_letters + w[i+1:]
            result.append(candidate)
        return result

    def replace_edits(self, w):
        result = []
        for i in range(len(w)):
            for c in self.ALPHABET:
                if c != w[i]:
                    candidate = w[:i] + c + w[i+1:]
                    result.append(candidate)
        return result

    def candidates(self, w, in_vocabulary=True):
        all_candidates = self.delete_edits(w) + self.insert_edits(w) + self.transpose_edits(w) + self.replace_edits(w)
        if in_vocabulary:
            all_candidates = [x for x in all_candidates if x in self.vocabulary]
        return set(all_candidates)

In [4]:
train_fname = 'data/corpus.txt'
candidate_model = CandidateModel(train_fname)

In [5]:
# candidate_model can be used to get the set of in-vocabulary candidate corrections for a spelling error.
# All candidate corrections are within edit distance one of the spelling error.
candidate_model.candidates('frend')

{'fiend', 'fred', 'freed', 'freud', 'friend', 'rend', 'trend'}

In [6]:
import math
EPS = 0.0001

# An unsmoothed unigram language model. You should read this to understand how it works.
# See examples below of how to use it.
class UnsmoothedUnigramLM:
    def __init__(self, fname):
        self.counts = {}
        self.train(fname)

    def train(self, fname):
        for line in open(fname):
            tokens = line.split()
            for t in tokens:
                self.counts[t] = self.counts.get(t, 0) + 1

        # Computing this sum once during training, instead of every
        # time it's needed in log_prob, speeds things up
        self.num_instances = sum(self.counts.values())

    def log_prob(self, word):
        # Compute probabilities in log space to avoid underflow errors
        # (This is not actually a problem for this language model, but
        # it can become an issue when we multiply together many
        # probabilities)
        if word in self.counts:
            return math.log(self.counts[word]) - math.log(self.num_instances)
        else:
            # This is a bit of a hack to get a float with the value of
            # minus infinity for words that have probability 0
            return float("-inf")

    # These methods might be helpful later for implementing Stupid Backoff
    def get_count(self, word):
        return self.counts.get(word, 0)

    def get_num_instances(self):
        return self.num_instances

    def check_probs(self):
        # Hint: Writing code to check whether the probabilities you
        # have computed form a valid probability distribution is very
        # helpful, particularly when you start incorporating smoothing
        # It can be a bit slow, however, especially for bigram language
        # models, so you might want to turn these checks off once
        # you're convinced things are working correctly.

        # Make sure the probability for each word is between 0 and 1
        for w in self.counts:
            assert 0 - EPS < math.exp(self.log_prob(w)) < 1 + EPS
        # Make sure that the sum of probabilities for all words is 1
        assert 1 - EPS < \
            sum([math.exp(self.log_prob(w)) for w in self.counts]) < \
            1 + EPS


In [7]:
# TODO Implement BigramLM following the explanation of the Bigram Language Model above by
# completing the constructor and log_prob methods. You are welcome to use additional methods
# as needed for your solution, but you should not change the signature for the constructor or
# log_prob or check_probs methods because other parts of the starter code (which you shouldn't
# modify) rely on these.

class BigramLM:
    def __init__(self, fname):
        # TODO Complete this method
        self.counts = {}
        self.bigram_counts = {}
        self.train(fname)

    def train(self, fname):
        for line in open(fname):
            tokens = line.split()
            for i, t in enumerate(tokens):
                self.counts[t] = self.counts.get(t, 0) + 1
                if i < len(tokens) - 1:
                  pair = (tokens[i], tokens[i+1])
                  self.bigram_counts[pair] = self.bigram_counts.get(pair, 0) + 1

        # Adding UNK to counts explicitly
        if "UNK" not in self.counts:
            self.counts["UNK"] = 0
        # Computing this sum once during training, instead of every
        # time it's needed in log_prob, speeds things up
        # copied the comments also
        self.num_instances = sum(self.counts.values())
        self.vocab = len(self.counts)

        #Print </s> count after training
        # print(f"Count of </s>: {self.counts.get('</s>', 0)}")

    def log_prob(self, w1, w2):
        # TODO Complete this method
        # This method should return log(P(w2|w1) (using add-1 smoothing)

        # Checking to see if the word is known or unknown
        w1 = w1 if w1 in self.counts else "UNK"
        w2 = w2 if w2 in self.counts else "UNK"

        # Getting counts safely (0 if not present)
        bigram_count = self.bigram_counts.get((w1, w2), 0)
        unigram_count = self.counts.get(w1, 0)

        # Applying Laplace smoothing
        return math.log((bigram_count + 1) / (unigram_count + self.vocab))

    # Unigram check_probs takes just a second or so but this takes a lot of time
    # def check_probs(self):
    #     # Hint: Writing code to check whether the probabilities you
    #     # have computed form a valid probability distribution is very
    #     # helpful, particularly when you start incorporating smoothing
    #     # It can be a bit slow, however, especially for bigram language
    #     # models, so you might want to turn these checks off once
    #     # you're convinced things are working correctly.

    #     for w1 in self.counts.keys():
    #       if w1 == "</s>":  # skip end-of-sentence as w1
    #           continue
    #       total_prob = 0.0
    #       for w2 in self.counts.keys():  # iterate over all possible w2
    #           p = math.exp(self.log_prob(w1, w2))
    #           assert 0 - EPS < p < 1 + EPS, f"Probability out of range: P({w2}|{w1})={p}"
    #           total_prob += p
    #       assert abs(total_prob - 1.0) < EPS, f"Probabilities for {w1} do not sum to 1 (sum={total_prob})"

In [8]:
# TODO Implement Stupid Backoff following the explanation of Stupid Backoff above.
# Note that you should have the various counts required to do this in unigram_lm
# and bigram_lm. As such, your implementation here should be very short. (My
# sample solution is about 4 lines. If you're writing a lot of code, you are
# likely off track.)

def stupid_backoff_score(w1, w2, lmbda):
    bigram_prob = bigram_lm.bigram_counts.get((w1, w2), 0) / bigram_lm.counts.get(w1, 1)
    unigram_prob = unigram_lm.counts.get(w2, 0) / unigram_lm.num_instances
    if bigram_prob > 0:
        return bigram_prob
    else:
        return (lmbda*unigram_prob)

In [9]:
# Create and train the language models
unigram_lm = UnsmoothedUnigramLM(train_fname)
bigram_lm = BigramLM(train_fname)

In [10]:
# You can comment out these lines out to run faster...
# unigram_lm.check_probs()
# bigram_lm.check_probs()

In [11]:
def get_predictions(predict_dataset_fname, model, stupid_backoff_lambda=1):
    # Get the predictions for a model for each instance in a dataset
    # model must be 1 of 'unigram', 'bigram', 'stupid'
    # stupid_backoff_alpha is only used when model = 'stupid'
    corrections = []
    for line in open(predict_dataset_fname):
        # Split the line on a tab; get the target word to correct and
        # the sentence it's in
        target_index,sentence = line.split('\t')
        target_index = int(target_index)
        sentence = sentence.split()
        target_word = sentence[target_index]

        # Get the in-vocabulary candidates
        iv_candidates = candidate_model.candidates(target_word)

        # Find the candidate correction with the highest probability;
        # if no candidate has non-zero probability, or there are no
        # candidates, give up and output the original target word as
        # the correction.
        best_prob = float('-inf')
        best_correction = target_word
        for ivc in sorted(iv_candidates):
            if model == 'unigram':
                unigram_log_prob = unigram_lm.log_prob(ivc)
                ivc_log_prob = unigram_log_prob
            elif model == 'bigram':
                bigram_log_prob = bigram_lm.log_prob(sentence[target_index - 1], ivc)
                bigram_log_prob += bigram_lm.log_prob(ivc, sentence[target_index + 1])
                ivc_log_prob = bigram_log_prob
            elif model == 'stupid':
                # Note that for stupid backoff we're not using log space
                ivc_log_prob = stupid_backoff_score(sentence[target_index - 1], ivc, stupid_backoff_lambda)
                ivc_log_prob *= stupid_backoff_score(ivc, sentence[target_index + 1], stupid_backoff_lambda)
            else:
                assert False
            if ivc_log_prob > best_prob:
                best_prob = ivc_log_prob
                best_correction = ivc
        corrections.append(best_correction)
    return corrections

In [12]:
def print_accuracy_for_predictions(predictions, keys):
    # If the length of the output and keys are not the same, something went
    # wrong...
    assert len(predictions) == len(keys)

    num_correct = 0
    total = 0
    for p,k in zip(predictions,keys):
        if p == k:
            num_correct += 1
        total += 1
    accuracy = num_correct / total
    print("Num correct: ", num_correct)
    print("Total: ", total)
    print("Accuracy:", round(accuracy, 3))


In [13]:
dev_fname = 'data/dev.txt'
unigram_dev_predictions = get_predictions(dev_fname, 'unigram')
bigram_dev_predictions = get_predictions(dev_fname, 'bigram')
# The third argument to get_predictions is the weight for stupid backoff
# (lambda in Eqation 3.31 in the textbook)
stupid_dev_predictions = get_predictions(dev_fname, 'stupid', 1)

dev_keys_fname = 'data/dev.keys.txt'
dev_keys = [x.strip() for x in open(dev_keys_fname)]

print('Unigram:')
print_accuracy_for_predictions(unigram_dev_predictions, dev_keys)

print()
print('Bigram:')
print_accuracy_for_predictions(bigram_dev_predictions, dev_keys)

print()
print('Stupid backoff:')
print_accuracy_for_predictions(stupid_dev_predictions, dev_keys)


Unigram:
Num correct:  175
Total:  421
Accuracy: 0.416

Bigram:
Num correct:  242
Total:  421
Accuracy: 0.575

Stupid backoff:
Num correct:  244
Total:  421
Accuracy: 0.58


The code above uses stupid backoff with a single value for lambda (1). Your task here is to find a good value for lambda. We will discuss approaches for doing this in lecture. Importantly, you must only consider the development data (and crucially not the test data) when doing so.

In [14]:
import numpy as np

best_lambda = None
best_accuracy = 0.0

for lmbda in np.arange(0.05, 1.05, 0.05):
    stupid_dev_predictions = get_predictions(dev_fname, 'stupid', lmbda)
    print(f'\nStupid backoff (λ={lmbda:.2f}):')
    # just printing the accuracy for visualization
    print_accuracy_for_predictions(stupid_dev_predictions, dev_keys)

    # Computing accuracy manually (since I can't modify the "print_accuracy_for_predictions" function)
    num_correct = sum([p == k for p, k in zip(stupid_dev_predictions, dev_keys)])
    accuracy = num_correct / len(dev_keys)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_lambda = lmbda

print(f"\nBest λ = {best_lambda:.2f} with Accuracy = {round(best_accuracy, 3)}")



Stupid backoff (λ=0.05):
Num correct:  245
Total:  421
Accuracy: 0.582

Stupid backoff (λ=0.10):
Num correct:  245
Total:  421
Accuracy: 0.582

Stupid backoff (λ=0.15):
Num correct:  245
Total:  421
Accuracy: 0.582

Stupid backoff (λ=0.20):
Num correct:  244
Total:  421
Accuracy: 0.58

Stupid backoff (λ=0.25):
Num correct:  246
Total:  421
Accuracy: 0.584

Stupid backoff (λ=0.30):
Num correct:  246
Total:  421
Accuracy: 0.584

Stupid backoff (λ=0.35):
Num correct:  246
Total:  421
Accuracy: 0.584

Stupid backoff (λ=0.40):
Num correct:  245
Total:  421
Accuracy: 0.582

Stupid backoff (λ=0.45):
Num correct:  246
Total:  421
Accuracy: 0.584

Stupid backoff (λ=0.50):
Num correct:  246
Total:  421
Accuracy: 0.584

Stupid backoff (λ=0.55):
Num correct:  245
Total:  421
Accuracy: 0.582

Stupid backoff (λ=0.60):
Num correct:  246
Total:  421
Accuracy: 0.584

Stupid backoff (λ=0.65):
Num correct:  245
Total:  421
Accuracy: 0.582

Stupid backoff (λ=0.70):
Num correct:  245
Total:  421
Accuracy:

In [15]:
# When you are done all of the parts above, evaluate the models on the test data by running this cell.
# Note that you should set the value of BEST_LAMBDA to whatever value you selected over the dev data above.
test_fname = 'data/test.txt'
unigram_test_predictions = get_predictions(test_fname, 'unigram')
bigram_test_predictions = get_predictions(test_fname, 'bigram')

BEST_LAMBDA = .25
stupid_test_predictions = get_predictions(test_fname, 'stupid', BEST_LAMBDA)

test_keys_fname = 'data/test.keys.txt'
test_keys = [x.strip() for x in open(test_keys_fname)]

print('Unigram:')
print_accuracy_for_predictions(unigram_test_predictions, test_keys)

print()
print('Bigram:')
print_accuracy_for_predictions(bigram_test_predictions, test_keys)

print()
print('Stupid backoff:')
print_accuracy_for_predictions(stupid_test_predictions, test_keys)


Unigram:
Num correct:  140
Total:  332
Accuracy: 0.422

Bigram:
Num correct:  179
Total:  332
Accuracy: 0.539

Stupid backoff:
Num correct:  196
Total:  332
Accuracy: 0.59


## Report

Write a report addressing at least the following points:

1. Compare the 3 models (unigram, bigram, and stupid backoff). Which model performs best? Is the relative performance of the models consistent across the development and test data?

1. For whichever model you find to perform best, why do you believe it performs better than the others? Justify your answer?

1. Clearly describe the process that you used to select the best value for lambda. What value of lambda did you find to work best?


#### **Report**


#### **1. Compare the 3 models (unigram, bigram, and stupid backoff). Which model performs best? Is the relative performance of the models consistent across the development and test data?**

I evaluated three language models — **Unigram**, **Bigram**, and **Stupid Backoff** — on both the development and test datasets.  
After tuning λ for the Stupid Backoff model, the best performance was achieved at **λ = 0.25**.  
The results are summarized below:

| Model           | Dev Accuracy | Test Accuracy |
|-----------------|-------------|--------------|
| **Unigram**     | 0.416       | 0.422        |
| **Bigram**      | 0.575       | 0.539        |
| **Stupid Backoff** (λ = 0.25) | **0.584** | **0.590** |

**Observations:**  
- The **Stupid Backoff model** achieves the highest accuracy on both development and test datasets.  
- The relative performance order (**Stupid Backoff > Bigram > Unigram**) remains **consistent across both datasets**, indicating that the improvement is robust.

---

#### **2. For whichever model you find to perform best, why do you believe it performs better than the others? Justify your answer?**

The Stupid Backoff model outperforms the others because:  

- It **uses bigram probabilities when available**, effectively capturing local word dependencies.  
- For unseen bigrams, it **backs off to a scaled unigram counting (denoted by S)**, preventing zero-probability issues.  
- This hybrid approach ensures better coverage for rare word pairs while still leveraging context for frequent ones.  

Thus, Stupid Backoff strikes the best balance between **context sensitivity** and **generalization**, resulting in superior accuracy.

---

#### **3. Clearly describe the process that you used to select the best value for lambda. What value of lambda did you find to work best?**

To select the best value of λ for the Stupid Backoff model, I performed a **grid search** over the interval **[0.05, 1.0]** with a step size of **0.05**.  
For each λ, I:

1. Computed predictions on the development set using the `get_predictions` function.
2. Measured accuracy using `print_accuracy_for_predictions`.
3. Compared the accuracy with the best accuracy found so far and updated the best λ if performance improved.
4. Though I found Multiple values yielding the highest accuracy but according to the logic I used(if the accuracy is not grater then it won't update the value of lambda) the initial value that got the highest score was chosen.

**Observations:**  

I observed that **multiple λ values** (0.25, 0.30, 0.35, 0.45, 0.50, 0.60) all achieved the **same highest accuracy of 0.584** on the development set.
This indicates that the Stupid Backoff model is relatively **insensitive to λ within this range**, as scaling the unigram probability slightly more or less does not significantly change which predictions are correct.

## What to submit

When you're done, submit this file to the assignment 1 dropbox on D2L. (You don't need to submit any of the data files we provided you with for this assignment).

## Grading

Your assignments will be graded based primarily on the correctness of their implementation and the written answers in the report.

Assignments that do not conform to the specifications outlined above might not be graded (e.g., modifying parts of the starter code that you were not asked to modify). Assignments that we are unable to run in a reasonable amount of time (less than one minute) also might not be graded. Grades will be out of 10 and broken down as follows:

- Bigram LM: 5

- Stupid Backoff: 2

- Report / discussion: 3
